### Домашнее задание 2

В этом задании вам необходимо:

1. Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind. 
2. Реализовать bootstrap для оценки (через доверительные интервалы) медианных и средних значений распределений. Рассчитать для распределений: нормальное, экспоненциальное, смесь нормальных
3. Рассчитать мощность критерия для t-критерия и критерия Манна-Уитни для разных распределений и разном эффекте. Отдельно изучить случай, когда средние равны, а дисперсии сильно отличаются.
4. Оценить корректность t-критерия и критерия Манна-Уитни на разных распределениях.

Задание принимается в jupyter notebook

In [353]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objs as go
from math import sqrt

### 1. T-критерий Стьюдента

In [363]:
def get_disp_val(a):
    return sum((a - a.mean())**2)/(len(a)-1)

In [398]:
def my_ttest(a_1, a_2, side='two-sided', alpha=0.05):
    #t-statistic
    statistic = (a_1.mean() - a_2.mean()) / sqrt(get_disp_val(a_1)/len(a_1) + get_disp_val(a_2)/len(a_2))
    
    #p-value
    p_value = sps.t.sf(np.abs(statistic), len(a_1)-1)
    
    if side == 'two-sided':
        p_value = p_value * 2
    if side == 'less':
        p_value = 1 - p_value if statistic > 0 else p_value
    if side == 'greater':
        p_value = p_value if statistic > 0 else 1 - p_value
        
    #make a decision
    null_hypothesis_rejection = p_value < alpha
        
    return {'statistic': statistic, 'pvalue': p_value, 'null_hypothesis_rejection': null_hypothesis_rejection}

#### Проверяем, что все работает
Спойлер - есть проблемы с точностью, но, кажется, сильно ни на что не влияет.
Это не полноценный тест. Чтобы убедиться, что работает с двух сторон, нужно поменять аргументы в sps.ttest_ind(rvs1, rvs2, alternative='less') и в my_ttest(rvs1, rvs2, side='less'). Возможно так же "greater"

In [404]:
rng = np.random.default_rng()

for i in range(100):
    rvs1 = sps.norm.rvs(loc=5, scale=10, size=500, random_state=rng)
    rvs2 = sps.norm.rvs(loc=5, scale=10, size=500, random_state=rng)
    
    lib_result = sps.ttest_ind(rvs1, rvs2, alternative='two-sided')
    my_result = my_ttest(rvs1, rvs2, side='two-sided')
    
    if round(my_result['statistic'], 6) != round(lib_result.statistic, 6):
        print('Something wrong with statistic')
        
    if round(my_result['pvalue'], 2) != round(lib_result.pvalue, 2):
        print(f'Something wrong with pvalue: \n my: {my_result["pvalue"]} lib: {lib_result.pvalue}')

Something wrong with pvalue: 
 my: 0.5151062383263785 lib: 0.514956336168312
Something wrong with pvalue: 
 my: 0.16506497483785823 lib: 0.16475515614501313
Something wrong with pvalue: 
 my: 0.155280727543502 lib: 0.15496834708540966
Something wrong with pvalue: 
 my: 0.1952088736572269 lib: 0.19490925877407778
Something wrong with pvalue: 
 my: 0.6450002265998096 lib: 0.6448998125127321


### 2. Bootstrap и доверительные интервалы

In [444]:
def get_bootstrap_sample_indices(sample_size: int, n_samples: int) -> np.ndarray:
    return np.random.randint(0, sample_size, (n_samples, sample_size))

def get_bootstrap_samples(X: np.ndarray, n_samples: int) -> np.ndarray:
    return X[get_bootstrap_sample_indices(len(X), n_samples)]

#### Доверительные интервалы для нормального распределения

In [526]:
def get_CI_bootstrap(distribution):
    median = np.median(distribution)
    mean = np.mean(distribution)
    
    X_bootstrap = get_bootstrap_samples(distribution, n_samples=10_000)
    
    metrics_median_ar = [np.median(X_bootstrap[i]) for i in range(len(X_bootstrap))]
    metrics_mean_ar = [np.mean(X_bootstrap[i]) for i in range(len(X_bootstrap))]
    
    alpha = 0.05
    q = 0.5
    c = sps.norm.ppf(1 - alpha/2)
    CI_mean = (metrics_mean - c * (np.std(X_bootstrap) / sqrt(len(X_bootstrap[0]))), metrics_mean + c * (np.std(X_bootstrap) / sqrt(len(X_bootstrap[0]))))
    CI_median = (len(X_bootstrap[0]) * q - c * sqrt(len(X_bootstrap[0]))*q*(1-q), len(X_bootstrap[0]) * q + c * sqrt(len(X_bootstrap[0]))*q*(1-q))
    
    return CI_mean, CI_median

In [529]:
size = 1000
x_norm = sps.norm.rvs(loc=4, scale=10, size=size)
distributions = [sps.norm.rvs(loc=4, scale=10, size=size),
                 sps.expon.rvs(loc=4, scale=10, size=size),
                 sps.norm.rvs(loc=4, scale=10, size=size) + sps.norm.rvs(loc=3, scale=5, size=size)]

In [530]:
for distr in distributions:
    print(get_CI_bootstrap(distr))

((array([3.44541736, 3.72676516, 3.62908879, ..., 3.68864453, 3.53727091,
       3.63130367]), array([4.74284264, 5.02419045, 4.92651407, ..., 4.98606981, 4.8346962 ,
       4.92872895])), (484.50512419238595, 515.494875807614))
((array([3.520829  , 3.8021768 , 3.70450042, ..., 3.76405617, 3.61268255,
       3.70671531]), array([4.66743101, 4.94877881, 4.85110243, ..., 4.91065817, 4.75928456,
       4.85331731])), (484.50512419238595, 515.494875807614))
((array([3.40168702, 3.68303483, 3.58535845, ..., 3.64491419, 3.49354058,
       3.58757333]), array([4.78657298, 5.06792078, 4.97024441, ..., 5.02980015, 4.87842653,
       4.97245929])), (484.50512419238595, 515.494875807614))


### 3. Мощность критериев

##### Общая функция

In [572]:
def get_criterion_power(criterion, distr, effect=1, scale=3, loc=10, alpha=0.05):
    size = 100
    n_exp = 10_000
    alpha = alpha
    effect = effect
        
    p_values = []
    for i in range(n_exp):
        x_a = distr(loc=loc, scale=scale, size=size)
        x_b = distr(loc=loc + effect, scale=scale, size=size)
        p_value = sps.ttest_ind(x_a, x_b).pvalue
        p_value = sps.mannwhitneyu(x_a, x_b).pvalue
        
        p_values.append(p_value)
    p_values = np.array(p_values)
    
    return p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100     

##### Параметры, которые будут перебираться в функции выше

In [573]:
distributions = {'norm': sps.norm.rvs,
                 'expon': sps.expon.rvs}

criterions = {'t-test': sps.ttest_ind, 
              'mannwhitneyu': sps.mannwhitneyu}

params = [dict(effect=1, scale=10, loc=10, alpha=0.1),
         dict(effect=5, scale=10, loc=10, alpha=0.1),
         dict(effect=1, scale=10, loc=10, alpha=0.01),
         dict(effect=1, scale=10, loc=1, alpha=0.01)]

##### Эксперименты

In [574]:
for i in distributions:
    print(f'distr: {i}')
    for j in criterions:
        for param in params:
            criterion_power = get_criterion_power(criterion=criterions[j], distr=distributions[i], **param)
            print(f'criterion: {j}, params: {param}, power: {criterion_power}')
    print()

distr: norm
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.1}, power: 17.86
criterion: t-test, params: {'effect': 5, 'scale': 10, 'loc': 10, 'alpha': 0.1}, power: 96.00999999999999
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.01}, power: 2.96
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 1, 'alpha': 0.01}, power: 2.91
criterion: mannwhitneyu, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.1}, power: 18.17
criterion: mannwhitneyu, params: {'effect': 5, 'scale': 10, 'loc': 10, 'alpha': 0.1}, power: 96.54
criterion: mannwhitneyu, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.01}, power: 2.93
criterion: mannwhitneyu, params: {'effect': 1, 'scale': 10, 'loc': 1, 'alpha': 0.01}, power: 2.73

distr: expon
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.1}, power: 31.509999999999998
criterion: t-test, params: {'effect': 5, 'scale': 10, 'loc': 10, 'alpha': 0.1}, power: 99.

### 4. Корректность критериев 

##### Общая функция

In [579]:
def get_criterion_pvalues(criterion, distr, effect=1, scale=3, loc=10, alpha=0.05):
    size = 100
    n_exp = 10_000
    alpha = alpha
    effect = effect
        
    p_values = []
    for i in range(n_exp):
        x_a = distr(loc=loc, scale=scale, size=size)
        x_b = distr(loc=loc + effect, scale=scale, size=size)
        p_value = sps.ttest_ind(x_a, x_b).pvalue
        p_value = sps.mannwhitneyu(x_a, x_b).pvalue
        
        p_values.append(p_value)
    p_values = np.array(p_values)
    
    return p_values 

##### Параметры, которые будут перебираться в функции выше

In [582]:
distributions = {'norm': sps.norm.rvs,
                 'expon': sps.expon.rvs}

criterions = {'t-test': sps.ttest_ind, 
              'mannwhitneyu': sps.mannwhitneyu}

params = [dict(effect=1, scale=10, loc=10, alpha=0.1),
         dict(effect=5, scale=10, loc=10, alpha=0.1),
         dict(effect=1, scale=10, loc=10, alpha=0.01),
         dict(effect=1, scale=10, loc=1, alpha=0.01)]

##### Эксперименты

In [584]:
for i in distributions:
    print(f'distr: {i}')
    for j in criterions:
        for param in params:
            pvalues = get_criterion_pvalues(criterion=criterions[j], distr=distributions[i], **param)
            print(f'criterion: {j}, params: {param}')
            probs = []
            x = [0.01 * idx for idx in range(101)]
            for idx in range(101):
                alpha_step = 0.01 * idx
                probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])

            fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
            fig.update_layout(height=600, width=600, title="Q-Q plot")
            fig = go.Figure([go.Histogram(x=p_values, xbins={"start":0, "end":1, "size": 0.1})])            

distr: norm
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.1}
criterion: t-test, params: {'effect': 5, 'scale': 10, 'loc': 10, 'alpha': 0.1}
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.01}
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 1, 'alpha': 0.01}
criterion: mannwhitneyu, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.1}
criterion: mannwhitneyu, params: {'effect': 5, 'scale': 10, 'loc': 10, 'alpha': 0.1}
criterion: mannwhitneyu, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.01}
criterion: mannwhitneyu, params: {'effect': 1, 'scale': 10, 'loc': 1, 'alpha': 0.01}
distr: expon
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.1}
criterion: t-test, params: {'effect': 5, 'scale': 10, 'loc': 10, 'alpha': 0.1}
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 10, 'alpha': 0.01}
criterion: t-test, params: {'effect': 1, 'scale': 10, 'loc': 1, 'alpha': 0.01}
